In [1]:
from simple_rag.chat.chat_openai import ChatOpenAI
from simple_rag.prompt.template import visual_language_generation
from simple_rag.utils import connect_to_database, insert_to_db

## Get Connection to Database

In [2]:
collection = connect_to_database()

🔥 Successfully Get Access Database


## Initialize Large Language Model Object

In [3]:
llm = ChatOpenAI()

## Case Study 1: System's prompt take a short instruct from user's prompt

In [4]:
response = llm.request(system_prompt=visual_language_generation, user_prompt="British shorthair")
insert_to_db(collection=collection, data=response)
response

## Case Study 2: System's prompt take a long instruct from user's prompt

In [5]:
structured_sample = {
    "id": "user_oriented_task_0",
    "motivation_app": "Grammarly",
    "instruction": "The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words.",
    "instances": [
        {
            "input": "If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.",
            "output": "If you have any questions about my rate or find it necessary to increase or decrease this project's scope, please let me know.",
        }
    ],
}

def gen_prompt_no_input(ins, outp):

    sys_prompt = "You are a helpful, precise but picky assistant for checking the quality of the answer to a given instruction."
    prompt_template = "[Instruction]\n{ins}\n\n[The Start of Answer]\n{outp}\n\n[The End of Answer]\n\n[System]\n{criteria}\n\n"
    criteria = (
        "We would like you to answer several questions related to the quality of the answer to the given instruction. \n"
        + "1. Why this answer is not good for the given instruction? Analyse based on the Helpfulness, Relevance, Accuracy, Level of Details, and Structure. \n"
        + "2. Based on the reason you provided, please generate a better answer while preserving the same content. To achieve that, you may want to adjust the level of details, add bullet points, or use comprehensive words, etc. The answer should be in the format of [Better Answer] your answer [End] \n"
    )
    prompt = prompt_template.format(ins=ins, outp=outp, criteria=criteria)
    return sys_prompt, prompt
system_prompt, user_prompt = gen_prompt_no_input(ins=structured_sample["instruction"], outp=structured_sample["instances"][0]["output"])
response = llm.request(system_prompt=system_prompt, user_prompt=user_prompt, max_tokens=2048)
insert_to_db(collection=collection, data=response)
response

👾 Warning: Failed to refine the output of LLM because: Extra data: line 1 column 2 (char 1)
🔥 Successfully Log Request to Database
